https://github.com/quantum-challenge/2019/blob/master/problems/how_to_calculate_quantum_cost_en.ipynb


### QUANTUM COSTS

In [10]:
from qiskit import QuantumCircuit, Aer, assemble, IBMQ, transpile
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex, plot_bloch_multivector
from qiskit.circuit import Gate
import qiskit.extensions as q_ext
from qiskit.quantum_info import Statevector
from qiskit.circuit.quantumregister import QuantumRegister
from qiskit.exceptions import QiskitError
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

from qiskit import QuantumCircuit, QuantumRegister,\
    ClassicalRegister,transpile, Aer, IBMQ,execute
from qiskit.test.mock import FakeBoeblingen
backend = FakeBoeblingen()


import matplotlib.pyplot as plt
import numpy as np
import math
from math import pi

In [11]:
y = QuantumRegister(1)
c = ClassicalRegister(2)

In [12]:
# Graph colouring
def reflection(qc,x):
    # Performs reflection around state |0>: adds a negative phase only to state |0>
    qc.x(x) #apply X to whole register
    qc.h(x[-1])#apply hadamard to last qubit
    qc.mcx(x[:-1],x[-1]) #apply multi control x gate
    qc.h(x[-1])
    qc.x(x)
def diffuser(qc,x):
    # Transforms superposition state |s> -> |0>, reflects about |0>
    # and transforms back to |s> again.
    qc.h(x)
    reflection(qc,x)
    qc.h(x)

In [13]:
# Aufgabe 1.7) Graphcoloring 4 Farben
nodes_with_colors = QuantumRegister(8)
edges_2 = QuantumRegister(3)
y_2 = QuantumRegister(1)
qc_2 = QuantumCircuit(nodes_with_colors, edges_2, y_2, c)

# Knoten 0 ist 00 -> passt
# Knoten 1 ist 01
qc_2.x(nodes_with_colors[3])
# Knoten 2 ist 10
qc_2.x(nodes_with_colors[4])
# Knoten 3 ist unbekannt
qc_2.h(nodes_with_colors[6])
qc_2.h(nodes_with_colors[7])
# y ist 1
qc_2.x(y_2)
# ... und dann in Superposition
qc_2.h(y_2)

# Kanten auf 1
for e in edges_2:
    qc_2.x(e)

def multi_color_oracle(qc, nodes_with_colors, edges_2, y):
    def set_edge(qc,nodes_with_colors, edges_2, n, m, e):
        qc.cx(nodes_with_colors[n], nodes_with_colors[m])
        qc.cx(nodes_with_colors[n + 1], nodes_with_colors[m + 1])
        qc.x(nodes_with_colors[m])
        qc.x(nodes_with_colors[m + 1])
        qc.ccx(nodes_with_colors[m], nodes_with_colors[m + 1], edges_2[e])
        qc.x(nodes_with_colors[m])
        qc.x(nodes_with_colors[m + 1])
        qc.cx(nodes_with_colors[n], nodes_with_colors[m])
        qc.cx(nodes_with_colors[n + 1], nodes_with_colors[m + 1])

    set_edge(qc, nodes_with_colors, edges_2, 0, 6, 0)
    set_edge(qc, nodes_with_colors, edges_2, 2, 6, 1)
    set_edge(qc, nodes_with_colors, edges_2, 4, 6, 2)
    qc.mcx(edges_2, y)
    set_edge(qc, nodes_with_colors, edges_2, 0, 6, 0)
    set_edge(qc, nodes_with_colors, edges_2, 2, 6, 1)
    set_edge(qc, nodes_with_colors, edges_2, 4, 6, 2)

for i in range(math.floor((math.pi / 4) * math.sqrt((2 ** 1) / 1))):
    multi_color_oracle(qc_2, nodes_with_colors, edges_2, y_2)
    diffuser(qc_2, [nodes_with_colors[6], nodes_with_colors[7]])

qc_2.measure(nodes_with_colors[6], c[0])
qc_2.measure(nodes_with_colors[7], c[1])

qc_2.draw(output='mpl', filename="graph_coloring_1.7.png", plot_barriers=False)

shots = 1000
# To use local qasm simulator
backend = Aer.get_backend('qasm_simulator')
job = execute(qc_2, backend=backend, shots=shots)
result = job.result()
data = result.get_counts()
print("Graph Coloring 1.7 Output:")
print(data)

# Wie verändert sich die Lösung, wenn Sie die Farbe von Knoten 2 noch nicht kennen?
# Hadamard auf die Qubits von Knoten 2, und entsprechend Kanten zwischen Knoten 1 und 2 und 2 und 0 ergänzen (analog wie bei Knoten 3),
# und Anzahl Iterationen anpassen (mit n = 2 statt 1)
# Output dann 4 Bits lang (Farben für beide Knoten)

pass_ = Unroller(['u3', 'cx', 'p'])
pm = PassManager(pass_)
new_circuit = pm.run(qc_2)
new_circuit.draw(output='mpl', filename="graph_coloring_1.7_transpiled.png")
ops = new_circuit.count_ops()
print(ops)
cost = ops['u3'] + ops['p'] + 10 * ops['cx']
print("Cost: " + str(cost))

Graph Coloring 1.7 Output:
{'11': 1000}
OrderedDict([('u3', 99), ('cx', 75), ('p', 15), ('measure', 2)])
Cost: 864
